In [1]:
require(tidyverse)
require(data.table)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [2]:
sscore_f <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/snpnet_PRS/HC_20190303/8_score_all/HC382.sscore'
sscore_df <- fread(
    cmd=paste0('cat ', sscore_f, ' | cut -f2,6'), sep='\t', data.table=F
)


In [5]:
phe_f <- '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/highconfidenceqc/phe/HC382.phe'
phe_df <- fread(
    cmd=paste0('cat ', phe_f, ' | cut -f2,3'), sep='\t', data.table=F
)
colnames(phe_df) <- c('IID', 'HC382')


In [31]:
combined_df <- sscore_df %>% rename(snpnet_PRS = SCORE1_AVG) %>% 
left_join(phe_df, by='IID') %>%
rename(outcome = HC382) %>% 
filter(outcome != -9) %>%
drop_na() %>%
mutate(Percentile = rank(-snpnet_PRS) / n())


In [32]:
combined_df %>% dim()

[1] 488374      4

In [34]:
cnt_middle <- combined_df %>% 
filter(0.4 <= Percentile, Percentile <= 0.6) %>% 
count(outcome) %>% rename(n_40_60 = n)

In [35]:
cnt_middle

outcome,n_40_60
1,87152
2,10522


In [96]:
compute_or <- function(df){
    joined_df <- df %>% count(outcome) %>% inner_join(cnt_middle, by='outcome') %>% gather(bin, cnt, -outcome)
    n_TP <- joined_df %>% filter(bin == 'n', outcome == 2) %>% select(cnt) %>% pull()
    n_TN <- joined_df %>% filter(bin != 'n', outcome == 1) %>% select(cnt) %>% pull()
    n_FP <- joined_df %>% filter(bin != 'n', outcome == 2) %>% select(cnt) %>% pull()
    n_FN <- joined_df %>% filter(bin == 'n', outcome == 1) %>% select(cnt) %>% pull()
    or <- (n_TP * n_TN) / (n_FP * n_FN)
    lor <- log(or)
    se_lor <- sqrt((1/n_TP) + (1/n_TN) + (1/n_FP) + (1/n_FN))
    lor <- exp(lor - 1.96 * se_lor)
    uor <- exp(lor + 1.96 * se_lor)
    c(or, lor, uor, n_TP / (n_TP + n_FN), n_TP, n_FN)
#     %>% print()    
}

In [99]:
compute_or_wrapper <- function(idx, df){
    bin_l = idx * 0.1
    bin_u = (idx + 1) * 0.1
    c(
        bin_l, bin_u, 
        df %>% filter(bin_l < Percentile, Percentile <= bin_u) 
        %>% compute_or()
    )    
}

In [100]:
res <- lapply(0:9, compute_or_wrapper, combined_df)

In [101]:
dff <- as.data.frame(res) 
colnames(dff) <- lapply(0:9, function(x){x * 10 + 5})


In [102]:
rownames(dff) <- c(
    'l', 'u', 'OR', 'OR_l', 'OR_u', 
    'Prevalence', 
    'n_cases', 'n_conts'
)

In [103]:
dff

,5,15,25,35,45,55,65,75,85,95
l,0.000000e+00,1.000000e-01,2.000000e-01,3.000000e-01,4.000000e-01,5.000000e-01,6.000000e-01,7.000000e-01,8.000000e-01,9.000000e-01
u,1.000000e-01,2.000000e-01,3.000000e-01,4.000000e-01,5.000000e-01,6.000000e-01,7.000000e-01,8.000000e-01,9.000000e-01,1.000000e+00
OR,3.757193e+00,2.073254e+00,1.576222e+00,1.259013e+00,1.085349e+00,9.161921e-01,8.041877e-01,6.944641e-01,5.670122e-01,4.350180e-01
OR_l,3.654001e+00,2.011964e+00,1.527283e+00,1.218016e+00,1.048743e+00,8.839145e-01,7.748165e-01,6.679848e-01,5.439712e-01,4.156770e-01
OR_u,3.971982e+01,7.705791e+00,4.753225e+00,3.494257e+00,2.953681e+00,2.508739e+00,2.252460e+00,2.027614e+00,1.795809e+00,1.585906e+00
Prevalence,3.120585e-01,2.001966e-01,1.598755e-01,1.319464e-01,1.158548e-01,9.959662e-02,8.849847e-02,7.735780e-02,6.407027e-02,4.989967e-02
n_cases,1.524000e+04,9.777000e+03,7.808000e+03,6.444000e+03,5.658000e+03,4.864000e+03,4.322000e+03,3.778000e+03,3.129000e+03,2.437000e+03
n_conts,3.359700e+04,3.906000e+04,4.103000e+04,4.239400e+04,4.317900e+04,4.397300e+04,4.451500e+04,4.506000e+04,4.570800e+04,4.640100e+04


In [104]:
dff %>% fwrite('HC382.csv', row.names = T, col.names=T)

In [105]:
combined_df %>% filter(
    .2 < Percentile, Percentile <= .3
)  %>% compute_or() %>% print()

[1] 1.576222e+00 1.527283e+00 4.753225e+00 1.598755e-01 7.808000e+03
[6] 4.103000e+04


In [85]:
combined_df %>% filter(
    .2 < Percentile, Percentile <= .3
) %>% count(outcome)

outcome,n
1,41030
2,7808


In [77]:
combined_df %>% filter(
    .2 < Percentile, Percentile <= .3
) %>%  count(outcome) %>% inner_join(cnt_middle, by='outcome') %>% gather(bin, cnt, -outcome)

outcome,bin,cnt
1,n,41030
2,n,7808
1,n_40_60,87152
2,n_40_60,10522
